# Crawling

> 상품정보
- 상품명
- 브랜드
- 품번
- Size info (사이즈 정보)
- Guide (가이드)
- 좋아요
- 태그
- 가격

> 리뷰정보
- 성별
- 키
- 몸무게
- 구매 사이즈(S, M, L, XL, ...)
- 사이즈(작다, 보통, 크다)
- 밝기
- 색감
- 두께감
- 리뷰내용

In [23]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import warnings
warnings.filterwarnings("ignore")

In [24]:
def extraction_review_info(browser):
    name_list = [] # 구매 의류
    size_buy_list = [] # 구매 사이즈

    sex_list = [] # 성별
    height_list = [] # 키
    weight_list = [] # 몸무게

    size_review_list = [] # 사이즈 리뷰
    brightness_list = [] # 밝기 리뷰
    color_list = [] # 색감 리뷰
    thickness_list = [] # 두께감 리뷰

    review_list = [] # 리뷰 내용

    # 현재 페이지 정보
    html = browser.page_source
    soup = BeautifulSoup(html,'lxml')

    # 구매한 의류, 사이즈
    merchandise_info = soup.find_all('div', 'review-goods-information__item')
    for i in range(len(merchandise_info)):
        try:
            name_list.append(merchandise_info[i].text.replace(" ", "").strip().replace("\n", ",").split(',')[0])
            size_buy_list.append(merchandise_info[i].text.replace(" ", "").strip().replace("\n", ",").split(',')[3])
        except:
            name_list.append("NaN")
            size_buy_list.append("NaN")
        
    # 고객 성별, 키, 몸무게(없을 수도 있음)
    customer_info = soup.find_all('div', 'review-profile__information')
    for i in range(len(customer_info)):
        try:
            sex_list.append(customer_info[i].text.strip().split('\n')[0].split(',')[0])
        except:
            sex_list.append("NaN")
            
        try:
            height_list.append(customer_info[i].text.strip().split('\n')[0].split(',')[1])
        except:
            height_list.append("NaN")
        
        try:
            weight_list.append(customer_info[i].text.strip().split('\n')[0].split(',')[2])
        except:
            weight_list.append("NaN")

    # 리뷰(선택) - 사이즈, 밝기, 색감, 두께감
    review_choice = soup.find_all('ul', 'review-evaluation__list')
    for i in range(len(review_choice)):
        try:
            size_review_list.append(review_choice[i].text.strip().replace("\n", ",").replace(" ", ",").split(",")[1])
            brightness_list.append(review_choice[i].text.strip().replace("\n", ",").replace(" ", ",").split(",")[3])
            color_list.append(review_choice[i].text.strip().replace("\n", ",").replace(" ", ",").split(",")[5])
            thickness_list.append(review_choice[i].text.strip().replace("\n", ",").replace(" ", ",").split(",")[7])
        except:
            size_review_list.append("NaN")
            brightness_list.append("NaN")
            color_list.append("NaN")
            thickness_list.append("NaN")  
            
    # 리뷰
    reviews = soup.find_all('div', "review-contents__text")
    for i in range(len(reviews)):
        review_list.append(reviews[i].text)

    return name_list, size_buy_list, sex_list, height_list, weight_list, size_review_list, brightness_list, color_list, thickness_list, review_list

In [25]:
print("#"*20, "Start", "#"*20)
browser = webdriver.Chrome()
# browser.maximize_window()

# 지금은 5개만 했지만 나중에 순서대로 선택하는 코드도 추가!
url_list = ['https://www.musinsa.com/app/goods/1108007', 
            'https://www.musinsa.com/app/goods/1558197', 
            'https://www.musinsa.com/app/goods/1283757', 
            'https://www.musinsa.com/app/goods/965917',
            'https://www.musinsa.com/app/goods/1114716']

browser.get(url_list[0])
print("browser open")
time.sleep(5)

# 들어가자마자 뜨는 할인쿠폰 창 없애기
try:
    button = browser.find_element_by_class_name('ab-close-button')
    button.send_keys(Keys.ENTER)
    print("coupon close")
except:
    print("no coupon")
time.sleep(2)

# 첫 페이지
page_cnt = 1
print(page_cnt, end=" ")
name_list, size_buy_list, sex_list, height_list, weight_list, size_review_list, brightness_list, color_list, thickness_list, review_list = extraction_review_info(browser)
time.sleep(2)

# 페이지 넘기면서 추출
for i in range(12):
    for i in range(4, 9 ,1):
        try:
            path = '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[' + str(i) + ']'
            button = browser.find_element_by_xpath(path)
            button.send_keys(Keys.ENTER)
            page_cnt += 1
            print(page_cnt, end=" ")
        except:
            print('except')
        time.sleep(2)
        
        # 다른 페이지 정보 받아서 한 리스트에 합치기
        name_list_1, size_buy_list_1, sex_list_1, height_list_1, weight_list_1, size_review_list_1, brightness_list_1, color_list_1, thickness_list_1, review_list_1 = extraction_review_info(browser)
        
        name_list.extend(name_list_1)
        size_buy_list.extend(size_buy_list_1)
        sex_list.extend(sex_list_1)
        height_list.extend(height_list_1)
        weight_list.extend(weight_list_1)
        size_review_list.extend(size_review_list_1)
        brightness_list.extend(brightness_list_1)
        color_list.extend(color_list_1)
        thickness_list.extend(thickness_list_1)
        review_list.extend(review_list_1)
        time.sleep(2)

print("\n", "#"*20, "Done", "#"*20)

# Webdriver 종료
browser.quit()

#################### Start ####################
browser open
coupon close
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 
 #################### Done ####################


In [28]:
review_df = pd.DataFrame({"name":name_list,
                          "size_buy":size_buy_list,
                          "sex":sex_list,
                          "height":height_list,
                          "weight":weight_list,
                          "size_review":size_review_list,
                          "brightness":brightness_list,
                          "color":color_list,
                          "thickness":thickness_list,
                          "review":review_list})

In [29]:
review_df

,name,size_buy,sex,height,weight,size_review,brightness,color,thickness,review
0,베이식블레이저[블랙],2XL,남성,177cm,92kg,보통이에요,밝아요,선명해요,보통이에요,생각보다 깔끔하게 스타일돼서 좋았고 사이즈는 팔만 좀 손목아래로 살짝 떨어지고 어깨...
1,베이식블레이저[블랙],2XL,남성,186cm,85kg,보통이에요,보통이에요,보통이에요,보통이에요,기본 블레이져로 좋습니다.장례식갈때 입을것같아요
2,베이식블레이저[블랙],L,남성,168cm,72kg,보통이에요,밝아요,선명해요,보통이에요,큰 기대 안했는데 재질이며 기장이며 너무 마음에 듭니다!
3,베이식블레이저[더스티베이지],L,남성,177cm,68kg,커요,어두워요,보통이에요,두꺼워요,레귤러사이즈인데 두께가있어서 그런지 좀 크네요.
4,베이식블레이저[차콜그레이],L,남성,176cm,75kg,보통이에요,보통이에요,보통이에요,보통이에요,"진짜 무탠다드는 사랑입니다.전투복, 결혼식 어디하나 빠지지않고다입을수있습니다.생각보..."
...,...,...,...,...,...,...,...,...,...,...
605,베이식블레이저[블랙],XL,남성,181cm,78kg,보통이에요,보통이에요,보통이에요,보통이에요,기본으로 무난하다고 해서 한번 구매해봤는데 생각보다 더 맘에듭니다
606,베이식블레이저[라이트베이지],M,남성,173cm,65kg,보통이에요,밝아요,선명해요,얇아요,세미핏으로 입을 수 있는 블레이저 검은색이 질리면 추천합니다
607,베이식블레이저[블랙],L,남성,170cm,60kg,보통이에요,보통이에요,보통이에요,보통이에요,소매긴거 말고는 아주 잘맞습니다 ㅎㅎ 색상도 기본 블랙이라 좋아요
608,베이식블레이저[차콜그레이],L,남성,170cm,60kg,보통이에요,보통이에요,보통이에요,보통이에요,소매기장이 조금길어서 수선했고 나머지는 딱맞습니다 !!


In [30]:
print(review_df.shape)
print(review_df.columns)

(610, 10)
Index(['name', 'size_buy', 'sex', 'height', 'weight', 'size_review',
       'brightness', 'color', 'thickness', 'review'],
      dtype='object')
